<a href="https://colab.research.google.com/github/dev-ago-uy/ExamenM-duloII.DM/blob/main/ExamenM%C3%B3duloII_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autor

Ing. Daniel Mendiola



**Objetivo**

Predecir el precio de casas utilizando un dataset provisto (disponible en GitHub). Aunque el dataset original contiene imágenes, no es necesario utilizarlas para este examen.

Fecha: Junio 2025

# Datos

x : https://raw.githubusercontent.com/AnIsAsPe/WA_ModuloII_DatosExamen/refs/heads/main/Datos/X.csv

y : https://raw.githubusercontent.com/AnIsAsPe/WA_ModuloII_DatosExamen/refs/heads/main/Datos/y.csv




# Contexto general



In [ ]:
# -----------------------------
# Carga de librerías necesarias
# -----------------------------

# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Manejo de valores faltantes
from sklearn.impute import SimpleImputer

# Configuraciones generales
import warnings
warnings.filterwarnings('ignore')

# Estilo visual
plt.style.use('seaborn-vcolor')
sns.set_theme(style="whitegrid")


# Nueva sección